### Step 1.

변수가 담기는 Variable 클래스를 구현한다. 

In [ ]:
import numpy as np

class Variable:
  def __init__(self, data):
    self.data = data

### Step 2.

Step 1에서 다루었던 Variable 클래스를 다루는 함수 클래스를 구현한다. 다음과 같은 2가지 원칙을 따른다.

* Function 클래스는 Variable 인스턴스를 입력받아 Variable 인스턴스를 출력한다.
* Variable 인스턴스의 실제 데이터는 인스턴스 변수인 data에 있다.

> __call__메서드는 파이썬의 특수 메서드이다. 이 메서드를 정의하면 f = Functinon() 형태로 함수의 인스턴스를 변수 f에 대입해두고, 나중에 f(...) 형태로 __call__ 메서드를 호출할 수 있다.

앞으로 구현할 함수는 이 Function 클래스를 상속하여, 구체화하기로 한다. 다음과 같은 2가지 원칙을 따른다.

* Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현하다.
* 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현한다.

In [ ]:
class Function:
  def __init__(self):
    pass

  # input은 Variable 클래스라고 가정한다.
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output

  def forward(self, x):
    raise NotImplementedError()

# Function 클래스를 상속받아, 기능을 구체화시킨다.
class Square(Function):
  def forward(self, x):
    return x ** 2

x = Variable(np.array(10))
f = Square()
y = f(x)
print(y.data)

100


### Step 4.

모델을 학습하는 것에 있어, 가장 중요한 것은 손실함수를 미분함으로써, 손실함수의 값을 최소화시키는 것이다. 이때 가장 쉽게 생각할 수 있는 미분법이 수치 미분(numerical differentiation) 이다. 보통 수치미분에서는 오차값을 가장 줄일 수 있는 중앙 차분(centered difference)을 많이 사용한다.

> 중앙차분 : (f(x + h) - f(x - h)) /  2*h

그러나 이러한 수치미분에는 치명적인 단점 2가지가 있는데, 다음과 같다.

* 수치 미분의 결과에는 오차가 포함되어 있는데, 경우에 따라 그 오차값이 매우 커질 수도 있다. 주로 '자릿수 누락' 때문인데, 중앙차분 등 '차이'를 구하는 계산은 주로 크기가 비슷한 값들을 다루므로 계산 결과에서 자릿수 누락이 생겨 유효 자릿수가 줄어들 수 있다.

* 수치 미분은 계산량이 많다. 신경망에서는 매개변수를 수백만개 이상 사용하는 건 일도 아니므로, 이 모두를 수치미분으로 구하는 것은 현실적이지 않다.

따라서, 수치미분은 보통 역전파를 정확하게 구현했는지 확인하는 용도로 사용된다. 이를 기울기 확인(gradient checking) 이라고 한다.

In [ ]:
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (eps * 2)

f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


### Step 6.

이번 단계에서는 Function 클래스를 발전시켜, Square, Exp 같은 세부적인 기능을 가진 함수가 Function 클래스를 상속받아 구현될 수 있도록 틀을 마련한다.

In [42]:
class Variable:
  def __init__(self, x):
    self.data = x
    self.grad = None # 역전파를 위해 Variable 클래스에 self.grad 속성을 추가

class Function:
  def __init__(self):
    self.input = None

  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    self.input = input
    return output

  def forward(self, x):
    return NotImplementedError()

  def backward(self, x):
    return NotImplementedError()

# square
class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.input.data
    gx = 2 * x * gy
    return gx

# exp
class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y

  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

# 역전파 구현
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


### Step 7.

이번 단계에서는 Define-by-Run을 활용하여, 역전파를 자동화하려고 한다.

> Define-by-Run이란 딥러닝에서 수행하는 계산들을 계산 시점에 '연결'하는 방식으로, '동적 계산 그래프'라고도 한다.

지금까지의 계산 그래프들은 모두 일직선으로 늘어선 계산이므로, 함수의 순서를 리스트 형태로 저장해두면 나중에 거꾸로 추척하는 식으로 역전파를 자동화할 수 있다. 그러나 분기가 있는 계싼 그래프나 같은 변수가 여러번 사용되는 복잡한 계산 그래프는 단순히 리스트로 저장하는 식으로 해결할 수 없다.

In [8]:
import numpy as np

class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

class Function:
  def __init__(self):
    self.input = None
    self.output = None

  # 계산 시점에서 함수와 변수의 관계를 저장한다. (Define-by-Run)
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

  def forward(self, x):
    return NotImplementedError()

  def backward(self, x):
    return NotImplementedError()

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

# 역전파 시행
y.grad = np.array(1.0)
C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


위의 미분 작업을 자동화할 수 있도록, Variable 클래스에 backward 메소드를 추가한다.

In [9]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  # Define-by-run 구현
  def backward(self):
    f = self.creator
    
    if f is not None:
      x = f.input
      x.grad = f.backward(self.grad)
      x.backward()

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### Step 8.

앞에서 Variable 클래스의 backward 메서드를 통해 Define-by-run 기법을 구현해 보았다. 앞의 backward 함수는 재귀적으로 backward 함수를 호출하여 역전파를 구현하는데, 이번 장에서는 이것을 반복문으로 대체해보겠다.

In [2]:
import numpy as np
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)
  
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### Step 9.

지금까지의 DeZero는 함수를 '파이썬 클래스'로 정의해 사용했다. 그래서 가령 Square 클래스를 사용하는 계산을 하려면 코드를 다음처럼 작성해야 했다.

```python
x = Variable(np.array(0.5))
f = Square()
y = f(x)
```

이번 단계에서는 3가지의 개선을 추가할 것인데 다음과 같다.

* 클래스인 Sqaure, Exp를 좀 더 함수 형식으로 사용할 수 있게 할 것.
```python
  def square(x):
    return Square()(x)
```
* Variable 클래스에 대입될 수 있는 data의 타입이 np.ndarray만 가능하도록 할 것.
```python
  if not isinstance(data, np.ndarray):
    raise TypeError('{}은 지원하지 않습니다.'.format(type(data)))
```
* 스칼라 형식의 넘파이 객체를 계산할 때, np.ndarray로 변환한 후 계산할 것.
```python
  def as_array(x):
    if np.isscalar(x):
      return np.array(x)
    return x
```

In [3]:
class Variable:
  def __init__(self, data):
    # 오로지 np.ndarray만 지원
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # 미분값이 없을 때, 자동으로 1.0을 할당
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)

class Function:
  def __init__(self):
    self.input = None
    self.output = None

  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(self.as_array(y))
    output.set_creator(self)
    self.input = input
    self.output = output
    return output
  
  # x가 스칼라이면 np.ndarray 타입으로 변경해준다.
  def as_array(self, x):
    if np.isscalar(x):
      return np.array(x)
    return x

  def forward(self, x):
    return NotImplementedError()

  def backward(self, x):
    return NotImplementedError()

def square(x):
  f = Square()
  return f(x)

def exp(x):
  f = Exp()
  return f(x)

x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### Step 10.

이번에는 지금까지 개발한 DeZero 기능들을 테스트해볼 것이다.

파이썬으로 테스트할 때는, 표준 라이브러리에 포함된 unittest를 사용하면 편하다. 아래 코드와 같이 unittest를 임포트하고, unittest.TestCase를 상속한 SquareTest 클래스를 구현한다. 이때, 테스트를 할 때는, 이름이 test로 시작하는 메서드를 만들고 그 안에 테스트할 내용을 적는다.

> 아래 코드에서는 square 함수의 출력이 기댓값과 같은지 확인하기 위해 self.assertEqual이라는 메서드를 사용했다. 이외에도, self.assertGreater, self.assertTrue 등 unittest에는 다양한 메서드가 준비되어 있다.

아래의 테스트 코드가 steps/step10.py 파일에 있다고 가정했을 때, 터미널에서 다음 명령을 실행하면 된다.

> $ python -m unittest steps/step10.py

In [10]:
import unittest

def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2*eps)

# Square 클래스를 테스트
class SquareTest(unittest.TestCase):
  # 테스트 메소드는 무조건 'test' 단어로 시작되어야 한다.
  def test_forward(self):
    x = Variable(np.array(2.0))
    y = Square(x)
    expected = np.array(4.0)
    self.assertEqual(y, expected)

  def test_backward(self):
    x = Variable(np.array(3.0))
    y = square(x)
    y.backward()
    expected = np.array(6.0)
    self.assertEqual(x.grad, expected)

  def test_gradient_check(self):
    x = Variable(np.random.rand(1))
    y = square(x)
    y.backward()
    num_grad = numerical_diff(f, x, eps=1e-4)
    flg = np.allclose(x.grad, num_grad)
    self.assertTrue(flg)